In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Capa convolucional 1: 32 filtros, kernel 3x3, activación ReLU
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # Capa de max pooling 1: tamaño del pool 2x2
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa convolucional 2: 64 filtros, kernel 3x3, activación ReLU
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # Capa de max pooling 2: tamaño del pool 2x2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa convolucional 3: 64 filtros, kernel 3x3, activación ReLU
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # Capa de max pooling 3: tamaño del pool 2x2
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Capa completamente conectada: 64 unidades, activación ReLU
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        # Capa de salida: 10 unidades (para clasificación en 10 clases), activación softmax
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # Aplicar la primera capa convolucional seguida de ReLU y pooling
        x = self.pool1(F.relu(self.conv1(x)))
        # Aplicar la segunda capa convolucional seguida de ReLU y pooling
        x = self.pool2(F.relu(self.conv2(x)))
        # Aplicar la tercera capa convolucional seguida de ReLU y pooling
        x = self.pool3(F.relu(self.conv3(x)))
        # Aplanar las características para la capa completamente conectada
        x = x.view(-1, 64 * 4 * 4)
        # Aplicar la primera capa completamente conectada seguida de ReLU
        x = F.relu(self.fc1(x))
        # Aplicar la capa de salida seguida de softmax
        x = F.softmax(self.fc2(x), dim=1)
        return x

# Crear una instancia de la red
net = CNN()

# Mostrar la arquitectura del modelo
print(net)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

# Definir la transformación para normalizar los datos
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Cargar los conjuntos de datos CIFAR-10 de entrenamiento y prueba
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Etiquetas de las clases
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Bucle de entrenamiento
for epoch in range(2):  # Recorrer el conjunto de datos varias veces
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # Poner a cero los gradientes de los parámetros
        optimizer.zero_grad()

        # Paso hacia adelante
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        # Paso hacia atrás y optimizar
        loss.backward()
        optimizer.step()

        # Imprimir estadísticas
        running_loss += loss.item()
        if i % 2000 == 1999:  # Imprimir cada 2000 mini-lotes
            print(f'[Epoch {epoch + 1}, Lote {i + 1}] pérdida: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Terminó el entrenamiento')

[Epoch 1, Lote 2000] pérdida: 2.302
[Epoch 1, Lote 4000] pérdida: 2.300
[Epoch 1, Lote 6000] pérdida: 2.280
[Epoch 1, Lote 8000] pérdida: 2.214
[Epoch 1, Lote 10000] pérdida: 2.189
[Epoch 1, Lote 12000] pérdida: 2.177
[Epoch 2, Lote 2000] pérdida: 2.163
[Epoch 2, Lote 4000] pérdida: 2.154
[Epoch 2, Lote 6000] pérdida: 2.139
[Epoch 2, Lote 8000] pérdida: 2.142
[Epoch 2, Lote 10000] pérdida: 2.121
[Epoch 2, Lote 12000] pérdida: 2.103
Terminó el entrenamiento


In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Exactitud en las 10000 imagenes de prueba: {100 * correct / total:.2f}%')

Exactitud en las 10000 imagenes de prueba: 36.70%
